In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns

In [2]:
datpath = '/Users/suhancho/data/Uniprot_metalbinding_challenge/'

In [3]:
os.listdir(datpath)

['POS_TRAIN.fasta',
 '.DS_Store',
 'NEG_TRAIN.fasta',
 'neg_sequence',
 'pos_sequence',
 'POS_TRAIN_FULL.fasta',
 'POS_TRAIN_FULL.tsv',
 'POS_TRAIN.tsv',
 'ChEBI-IDs_for_metal_binding.tsv',
 'test-dataset.fasta',
 'UniProt Metal Binding Site Challenge.docx',
 'wget-log']

In [4]:
def getdata(datpath):
    train_pos = pd.read_table(datpath+'POS_TRAIN_FULL.tsv')
    chebi = pd.read_table(datpath+'ChEBI-IDs_for_metal_binding.tsv')
    return(train_pos,chebi)

In [42]:
train_pos,chebi = getdata(datpath)

In [52]:
def get_label(accession,inpath,mode,ion):
    if mode=='positive':
        sequence_fasta = open(inpath+'/pos_sequence/'+accession+'.fasta','r').readlines()[0]
        bindset = train_pos[(train_pos.Accession==accession) & (train_pos['ChEBI-ID']==ion)]
        if len(bindset)!=0:
            label = np.zeros(len(sequence_fasta),dtype = int)
            for i in range(len(bindset)):
                pos = bindset.Position.tolist()[i]
                label[pos] = 1
            label = ''.join(label.astype(str))
        else : 
            label = np.NaN

    elif mode=='negative':
        sequence_fasta = open(inpath+'/neg_sequence/'+accession+'.fasta','r').readlines()[0]
        label = '0'*len(sequence_fasta)
        ion=None
    return(sequence_fasta,label)

In [53]:
neg_list = os.listdir(datpath+'/neg_sequence/')
pos_list = os.listdir(datpath+'/pos_sequence/')

In [54]:
get_label(neg_list[0].split('.')[0],datpath,'negative','CHEBI:29105')

('QPIPIVAIVALVVAIIIAIVVWSIVIIEYRKILRQRKIDRLIDRLIERAEDSGNESEGEISALVEMGVEMGHHAPWDVDDL',
 '000000000000000000000000000000000000000000000000000000000000000000000000000000000')

In [55]:
get_label('A7FLX7',datpath,'positive','CHEBI:60240')

('MRIGHGFDVHKFGENGSGPLIIGGVRIPYEKGLLAHSDGDVALHAATDALLGAAALGDIGKLFPDTDPAFKGADSRGLLREAYRRILAKGYKLGNLDITIIAQAPKMAPHIPQMRVNLAEDLQCHMDDINVKATTTEQLGFTGRGEGIACEAVVLLVNVEQG',
 '000000001010000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000')

In [56]:
train_pos_selected = train_pos[['Accession','ChEBI-ID']].drop_duplicates()

In [57]:
train_pos_selected['ChEBI-ID'].nunique()

29

In [79]:
oh_encoded_ion = pd.get_dummies(train_pos['ChEBI-ID'].unique())
oh_encoded_ion = oh_encoded_ion.sort_values(oh_encoded_ion.columns.tolist())[::-1]
oh_encoded_ion.reset_index(inplace = True,drop = True)

In [58]:
for i in range(10):
    print(get_label(train_pos_selected.Accession.tolist()[i],datpath,'positive',train_pos_selected['ChEBI-ID'].tolist()[i]))

('...')
